# Meta-ness

Julia allows us to talk in a "meta" way ("one level up"), about Julia code,  that is to "treat code as data" and manipulate it as just another object in Julia. (This is very similar to Lisp.)

One of the most basic objects in this approach are unevaluated *symbols*:

In [ ]:
:a  # "the symbol a"

In [ ]:
typeof(:a)

`:a` refers to the symbol `a`. We can evaluate it with the `eval` function:

In [ ]:
eval(:a)

`a` must be defined for this to work:

In [ ]:
a = 3

In [ ]:
eval(:a)

The `eval` function takes an expression and evaluates it, that is, *generates the corresponding code*

Every name for a thing is a symbol:

In [ ]:
:+, :sin

In [ ]:
typeof(:+)

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [ ]:
ex = :(a + b)  # the expression 'a + b'

In [ ]:
typeof(ex)

In [ ]:
b = 7
eval(ex)

An expression is just a Julia object, so we can introspect (find out information about it):

In [ ]:
dump(ex)

In [ ]:
dump(:(x = 3))

In [ ]:
Meta.@dump x = 3

In [ ]:
# ex.<TAB>

In [ ]:
ex.head

In [ ]:
ex.args

In [ ]:
@code_typed 1 + 2.5

In [ ]:
dump(ans)

More complicated expressions are represented as "abstract syntax trees" (ASTs), consisting of expressions nested inside expressions:

In [ ]:
ex = :( sin(3a + 2b^2) )

In [ ]:
ex.args

In [ ]:
typeof(ex.args[2])

In [ ]:
ex.args[2].args

In [ ]:
ex.args[1] = :cos

In [ ]:
ex

In [ ]:
blk = quote
    println("Hello")
end

In [ ]:
eval(blk)

In [ ]:
push!(blk.args, :(println("AFTER")))

In [ ]:
blk

In [ ]:
eval(blk)

In [ ]:
pushfirst!(blk.args, :(println("BEFORE")))

In [ ]:
eval(blk)

Expressions can be arbitrary Julia code that when evaluated will have side effects. For longer blocks of code, `quote...end` may be used instead of `:( ... )`

In [ ]:
ex2 = quote
    y = 3
    z = sin(y+1)
end

In [ ]:
eval(ex2)

In [ ]:
y, z

The full form of the abstract syntax tree in a style similar to a Lisp s-expression can be obtained using functions from the `Meta` module in `Base`:

In [ ]:
Meta.show_sexpr(ex2)

Another way of seeing the structure is with `dump`:

In [ ]:
dump(ex2)

# Macros

With the ability to think of code in terms of a data structure in the Julia language, we can now *manipulate* those data structures, allowing us to *create Julia code on the fly from within Julia*. This is known as *metaprogramming*: programming a program.

The name *macro* is given to a kind of "super-function" that takes a piece of code as an argument, and returns an altered piece of code. A macro is thus a very different kind of object than a standard function. [Although it can be thought of as a function in the mathematical sense of the word.]

The [Julia manual](http://julia.readthedocs.org/en/latest/manual/metaprogramming/) puts it like this: 

> macros map a tuple of argument *expressions* to a returned
> *expression*

In [ ]:
# @mac a b c ===>  code to execute

Although metaprogramming is possible in many languages, often by splicing strings together, Julia makes it considerably more natural, although not exactly easy – there seems to be something inherently tricky about the metaness of it.

Metaprogramming is useful in a variety of settings. The following are a few example use cases:
- to eliminate boilerplate (repetitive) code
- to automatically generate complex code that would be painful by hand
- to unroll loops for efficiency
- to inline code for efficiency



Macros are invoked using the `@` sign, e.g.

In [ ]:
@time sin(10)

A trivial example of defining a macro is the following, which runs whatever code it is passed two times. The `$` sign is used to interpolate the value of the expression (similar to its usage for string interpolation):

In [ ]:
macro twice(ex)
    quote
        $(esc(ex))
        $(esc(ex))
    end
end

In [ ]:
x = 0
@twice println(x += 1)

In [ ]:
ex = :(@twice println(x += 1))

In [ ]:
eval(ex)

In [ ]:
typeof(ex)

We can see what effect the macro actually has using `Meta.@macroexpand`:

In [ ]:
Meta.@macroexpand @twice println(x += 1)

In [ ]:
Meta.@macroexpand @time sin(10)

In [ ]:
macro mytime(ex)
    quote
        t0 = time()
        val = $ex
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

In [ ]:
@mytime (sleep(1); "done")

In [ ]:
Meta.@macroexpand @mytime (sleep(1); "done")

## Macro Hygiene (and lack thereof)

In [ ]:
t1 = "the first terminator movie"
@mytime t1 *= " is the best"

In [ ]:
Meta.@macroexpand @mytime t1 *= " is the best"

Oops! We don't want the macro to rewrite our `t1` to `#42#t1`. To prevent this, we can call the `esc` function on it in our macro:

In [ ]:
macro mytime(ex)
    quote
        t0 = time()
        val = $(esc(ex))
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

In [ ]:
t1 = "the first terminator movie"
@mytime t1 *= " is the best"

In [ ]:
Meta.@macroexpand @mytime t1 *= " is the best"

In [ ]:
macro set_a(val)
    :($(esc(:a)) = $(esc(val)))
end

In [ ]:
Meta.@macroexpand @set_a 1.5

In [ ]:
@set_a 1.5

In [ ]:
a

----
**Exercise**: Define a macro `@until` that does an `until` loop.

----

In [ ]:
# implement `@until` here

In [ ]:
let i = 0
    @until i==10 begin
        println(i)
        i += 1
    end
end

## Macros for numerical performance: Horner's method

There are many interesting examples of macros in `Base`. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$

may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a `for` loop. But if we know the polynomial *at compile time*, this loop may be unrolled using metaprogramming. This is implemented in the `Math` module in `math.jl` in `Base`, so the name of the macro which is not exported is `@Base.Math.horner`, so in the current namespace, `horner` should be undefined:

In [ ]:
# copied from base/math.jl
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

This is called as follows: to evaluate the polynomial $p(x) = 2 + 3x + 4x^2$ at $x=3$, we do

In [ ]:
x = 3
@horner(x, 2, 3, 4, 5)

To see what the macro does to this call, we again use `macroexpand`:

In [ ]:
Meta.@macroexpand @horner(x, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [ ]:
Meta.@macroexpand @Base.Math.horner(x, 2, 3, 4, 5)

In [ ]:
?muladd

In [ ]:
?fma

In [ ]:
f(x) = Base.Math.@horner(x, 1.2, 2.3, 3.4, 4.5)

In [ ]:
@code_llvm f(0.1)

In [ ]:
@code_native f(0.1)